# Import Useful Modules 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import gmtime, strftime
import time
import datetime
from collections import Counter
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Embedder
from gensim.models import FastText

# Classifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.grid_search import GridSearchCV as GS

import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

from preprocessing_pipeline import preprocessing

D:\Program\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversi

# ---------------------------------------------------------------------------------------------------------------

# Read Data To Pandas Dataframe

In [2]:
# data untuk klasifikasi kategori produk
large_data_for_classification=pd.read_csv("data/big.csv",header=None)
large_data_for_classification.dropna(axis=0,inplace=True)

# data untuk word embedding
data_for_embedding=pd.read_fwf('data/products2m.txt',header=None)
data_for_embedding["Product Title"]=data_for_embedding[0]
data_for_embedding=data_for_embedding[["Product Title"]]
data_for_embedding.dropna(inplace=True,axis=0)

In [3]:
data_for_embedding.head()

,Product Title
0,"PC LENOVO IC300s-i5(4460)-VGA with LED-19,5"
1,prosessor intel core i5 4440
2,LENOVO All in One aio310-0kid
3,PC Lenovo aio510-crid
4,HP Pavilion 570-P034D Win 10 Home


# ---------------------------------------------------------------------------------------------------------------

# Preprocessing

In [4]:
def remove_parentheses_old(input_string):
    result_string=input_string.lower()
    target_parentheses=['-','/','[',']','!','(',')',',','.','+','-',"'",'"',"|","*","@","#","!","<",">",":",";","?"]
    for parentheses in target_parentheses:
        result_string=result_string.replace(parentheses, ' ')
    result_string=result_string.strip(' ').split()
    return result_string

In [5]:
def remove_parentheses(input_string):
    input_string=''.join(i for i in input_string if not i.isdigit())
    result_string=input_string.lower()
    target_parentheses=['-','/','[',']','!','(',')',',','.','+','-',"'",'"',"|","*","@","#","!","<",">",":",";","?"]
    for parentheses in target_parentheses:
        result_string=result_string.replace(parentheses, ' ')
    result_string=result_string.strip(' ').split()
    return result_string

In [6]:
# menghapus karakter tidak penting dari data
product_title=[remove_parentheses(value) for value in data_for_embedding["Product Title"]]

# ---------------------------------------------------------------------------------------------------------------

# Finding Best Embedding Window

In [60]:
def accuracy(predicted,truth):
    result=[int(value) for value in np.array(predicted)==np.array(truth)]
    return sum(result)/len(result)

In [61]:
sampled_data=large_data_for_classification.sample(n=50000,random_state=1387178)
mask = np.random.rand(len(sampled_data)) < 0.8
train = sampled_data[mask]
validation = sampled_data[~mask]

In [63]:
result=[]
for EMBEDDING_WINDOW in [1,2,3,5,7]:
    print("TESTING ON EMBEDDING WINDOW OF {} | {}".format(EMBEDDING_WINDOW,str(datetime.datetime.now())))
    word_embedder = FastText(product_title, size=100, window=EMBEDDING_WINDOW, min_count=10, workers=4, sg=1, seed=SEED, min_n=5, iter=1)
    print("\tEMBEDDER CONSTRUCTED | {}".format(str(datetime.datetime.now())))
    preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
    embedded_data,label_encoder=preprocessor.preprocess_data(train[1],train[0])
    validation_set,validation_label_encoder=preprocessor.preprocess_data(validation[1],validation[0])
    embedded_data["sum"]=embedded_data.drop(["Labels"],axis=1).sum(axis=1)
    embedded_data=embedded_data.loc[embedded_data["sum"]!=0].drop("sum",axis=1)
    print("\tPREPROCESSING FINISHED | {}".format(str(datetime.datetime.now())))
    
    print("\n\tTRAINING CLASSIFIER | {}".format(str(datetime.datetime.now())))
    model = Sequential()
    model.add(Dense(2000, input_shape=(100,), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1500, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(107, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    history = model.fit(embedded_data.drop("Labels",axis=1),to_categorical(embedded_data["Labels"]), epochs=4, batch_size=32, shuffle=True)
    
    truth=[np.argmax(value) for value in to_categorical(validation_set["Labels"])]
    pred=[np.argmax(value) for value in model.predict(validation_set.drop("Labels",axis=1))]
    result.append(accuracy(pred,truth))
    print("\tVALIDATION ACCURACY : {}\n\n".format(accuracy(pred,truth)))

TESTING ON EMBEDDING WINDOW OF 1 | 2018-06-28 14:00:30.220999
	EMBEDDER CONSTRUCTED | 2018-06-28 14:01:17.235800
	PREPROCESSING FINISHED | 2018-06-28 14:01:21.980418

	TRAINING CLASSIFIER | 2018-06-28 14:01:21.981420
Epoch 1/4
39471/39471 [==============================] - 88s 2ms/step - loss: 2.6113
Epoch 2/4
39471/39471 [==============================] - 88s 2ms/step - loss: 2.2238
Epoch 3/4
39471/39471 [==============================] - 93s 2ms/step - loss: 2.0384
Epoch 4/4
39471/39471 [==============================] - 88s 2ms/step - loss: 1.8866
	VALIDATION ACCURACY : 0.1553686293913904

TESTING ON EMBEDDING WINDOW OF 2 | 2018-06-28 14:07:22.443479
	EMBEDDER CONSTRUCTED | 2018-06-28 14:08:24.282982
	PREPROCESSING FINISHED | 2018-06-28 14:08:31.842052

	TRAINING CLASSIFIER | 2018-06-28 14:08:31.842052
Epoch 1/4
39471/39471 [==============================] - 89s 2ms/step - loss: 2.5368
Epoch 2/4
39471/39471 [==============================] - 91s 2ms/step - loss: 2.1423
Epoch 3/4
394

In [67]:
result=[]
for EMBEDDING_WINDOW in [11]:
    print("TESTING ON EMBEDDING WINDOW OF {} | {}".format(EMBEDDING_WINDOW,str(datetime.datetime.now())))
    word_embedder = FastText(product_title, size=100, window=EMBEDDING_WINDOW, min_count=10, workers=4, sg=1, seed=SEED, min_n=5, iter=1)
    print("\tEMBEDDER CONSTRUCTED | {}".format(str(datetime.datetime.now())))
    preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
    embedded_data,label_encoder=preprocessor.preprocess_data(train[1],train[0])
    validation_set,validation_label_encoder=preprocessor.preprocess_data(validation[1],validation[0])
    embedded_data["sum"]=embedded_data.drop(["Labels"],axis=1).sum(axis=1)
    embedded_data=embedded_data.loc[embedded_data["sum"]!=0].drop("sum",axis=1)
    print("\tPREPROCESSING FINISHED | {}".format(str(datetime.datetime.now())))
    
    print("\n\tTRAINING CLASSIFIER | {}".format(str(datetime.datetime.now())))
    model = Sequential()
    model.add(Dense(2000, input_shape=(100,), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1500, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(107, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    history = model.fit(embedded_data.drop("Labels",axis=1),to_categorical(embedded_data["Labels"]), epochs=4, batch_size=32, shuffle=True)
    
    truth=[np.argmax(value) for value in to_categorical(validation_set["Labels"])]
    pred=[np.argmax(value) for value in model.predict(validation_set.drop("Labels",axis=1))]
    result.append(accuracy(pred,truth))
    print("\tVALIDATION ACCURACY : {}\n\n".format(accuracy(pred,truth)))

TESTING ON EMBEDDING WINDOW OF 11 | 2018-06-28 14:37:24.287958
	EMBEDDER CONSTRUCTED | 2018-06-28 14:38:29.156665
	PREPROCESSING FINISHED | 2018-06-28 14:38:33.982530

	TRAINING CLASSIFIER | 2018-06-28 14:38:33.983514
Epoch 1/4
39471/39471 [==============================] - 90s 2ms/step - loss: 2.4246
Epoch 2/4
39471/39471 [==============================] - 97s 2ms/step - loss: 2.0138
Epoch 3/4
39471/39471 [==============================] - 93s 2ms/step - loss: 1.8194
Epoch 4/4
39471/39471 [==============================] - 92s 2ms/step - loss: 1.6497
	VALIDATION ACCURACY : 0.17219198416625434




# ---------------------------------------------------------------------------------------------------------------

# Finding Best Embedding Dimension

In [7]:
def accuracy(predicted,truth):
    result=[int(value) for value in np.array(predicted)==np.array(truth)]
    return sum(result)/len(result)

In [8]:
sampled_data=large_data_for_classification.sample(n=50000,random_state=1387178)
mask = np.random.rand(len(sampled_data)) < 0.8
train = sampled_data[mask]
validation = sampled_data[~mask]

In [11]:
result=[]
for EMBEDDING_DIMENSION in [200]:
    print("TESTING ON EMBEDDING DIMENSION OF {} | {}".format(EMBEDDING_DIMENSION,str(datetime.datetime.now())))
    word_embedder = FastText(product_title, size=EMBEDDING_DIMENSION, window=7, min_count=10, workers=4, sg=1, seed=123972, min_n=5, iter=1)
    print("\tEMBEDDER CONSTRUCTED | {}".format(str(datetime.datetime.now())))
    preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
    embedded_data,label_encoder=preprocessor.preprocess_data(train[1],train[0])
    validation_set,validation_label_encoder=preprocessor.preprocess_data(validation[1],validation[0])
    embedded_data["sum"]=embedded_data.drop(["Labels"],axis=1).sum(axis=1)
    embedded_data=embedded_data.loc[embedded_data["sum"]!=0].drop("sum",axis=1)
    print("\tPREPROCESSING FINISHED | {}".format(str(datetime.datetime.now())))
    
    print("\n\tTRAINING CLASSIFIER | {}".format(str(datetime.datetime.now())))
    model = Sequential()
    model.add(Dense(2000, input_shape=(EMBEDDING_DIMENSION,), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1500, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(108, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    history = model.fit(embedded_data.drop("Labels",axis=1),to_categorical(embedded_data["Labels"]), epochs=2, batch_size=32, shuffle=True)
    
    truth=[np.argmax(value) for value in to_categorical(validation_set["Labels"])]
    pred=[np.argmax(value) for value in model.predict(validation_set.drop("Labels",axis=1))]
    result.append(accuracy(pred,truth))
    print("\tVALIDATION ACCURACY : {}\n\n".format(accuracy(pred,truth)))

TESTING ON EMBEDDING DIMENSION OF 200 | 2018-06-28 15:49:15.826871
	EMBEDDER CONSTRUCTED | 2018-06-28 15:50:45.477079
	PREPROCESSING FINISHED | 2018-06-28 15:50:54.013261

	TRAINING CLASSIFIER | 2018-06-28 15:50:54.013261
Epoch 1/2
39360/39360 [==============================] - 110s 3ms/step - loss: 2.4499
Epoch 2/2
39360/39360 [==============================] - 96s 2ms/step - loss: 2.0552
	VALIDATION ACCURACY : 0.05413328121946453




In [13]:
history = model.fit(embedded_data.drop("Labels",axis=1),to_categorical(embedded_data["Labels"]),initial_epoch=2, epochs=4, batch_size=32, shuffle=True)

Epoch 3/4
39360/39360 [==============================] - 95s 2ms/step - loss: 1.8584
Epoch 4/4
39360/39360 [==============================] - 96s 2ms/step - loss: 1.6978


# ---------------------------------------------------------------------------------------------------------------

# Finding Best Embedding Epoch

In [14]:
def accuracy(predicted,truth):
    result=[int(value) for value in np.array(predicted)==np.array(truth)]
    return sum(result)/len(result)

In [15]:
sampled_data=large_data_for_classification.sample(n=50000,random_state=1387178)
mask = np.random.rand(len(sampled_data)) < 0.8
train = sampled_data[mask]
validation = sampled_data[~mask]

In [18]:
result=[]
for EMBEDDING_EPOCH in [5,10,20]:
    print("TESTING ON EMBEDDING EPOCH OF {} | {}".format(EMBEDDING_EPOCH,str(datetime.datetime.now())))
    word_embedder = FastText(product_title, size=100, window=7, min_count=10, workers=4, sg=1, seed=132984, min_n=5, iter=EMBEDDING_EPOCH)
    print("\tEMBEDDER CONSTRUCTED | {}".format(str(datetime.datetime.now())))
    preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
    embedded_data,label_encoder=preprocessor.preprocess_data(train[1],train[0])
    validation_set,validation_label_encoder=preprocessor.preprocess_data(validation[1],validation[0])
    embedded_data["sum"]=embedded_data.drop(["Labels"],axis=1).sum(axis=1)
    embedded_data=embedded_data.loc[embedded_data["sum"]!=0].drop("sum",axis=1)
    print("\tPREPROCESSING FINISHED | {}".format(str(datetime.datetime.now())))
    
    print("\n\tTRAINING CLASSIFIER | {}".format(str(datetime.datetime.now())))
    model = Sequential()
    model.add(Dense(2000, input_shape=(100,), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1500, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(108, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    history = model.fit(embedded_data.drop("Labels",axis=1),to_categorical(embedded_data["Labels"]), epochs=2, batch_size=32, shuffle=True)
    
    truth=[np.argmax(value) for value in to_categorical(validation_set["Labels"])]
    pred=[np.argmax(value) for value in model.predict(validation_set.drop("Labels",axis=1))]
    result.append(accuracy(pred,truth))
    print("\tVALIDATION ACCURACY : {}\n\n".format(accuracy(pred,truth)))

TESTING ON EMBEDDING EPOCH OF 5 | 2018-06-28 16:00:47.446773
	EMBEDDER CONSTRUCTED | 2018-06-28 16:06:00.536481
	PREPROCESSING FINISHED | 2018-06-28 16:06:06.707894

	TRAINING CLASSIFIER | 2018-06-28 16:06:06.707894
Epoch 1/2
39641/39641 [==============================] - 93s 2ms/step - loss: 2.1676
Epoch 2/2
39641/39641 [==============================] - 96s 2ms/step - loss: 1.7120
	VALIDATION ACCURACY : 0.05561132302750452


TESTING ON EMBEDDING EPOCH OF 10 | 2018-06-28 16:09:20.115675
	EMBEDDER CONSTRUCTED | 2018-06-28 16:20:11.609734
	PREPROCESSING FINISHED | 2018-06-28 16:20:17.795171

	TRAINING CLASSIFIER | 2018-06-28 16:20:17.795171
Epoch 1/2
39641/39641 [==============================] - 89s 2ms/step - loss: 2.1154
Epoch 2/2
39641/39641 [==============================] - 89s 2ms/step - loss: 1.6391
	VALIDATION ACCURACY : 0.052198353744228064


TESTING ON EMBEDDING EPOCH OF 20 | 2018-06-28 16:23:20.202106
	EMBEDDER CONSTRUCTED | 2018-06-28 16:45:05.510317
	PREPROCESSING FINISHED

In [19]:
history = model.fit(embedded_data.drop("Labels",axis=1),to_categorical(embedded_data["Labels"]), initial_epoch=2,epochs=4, batch_size=32, shuffle=True)

Epoch 3/4
39641/39641 [==============================] - 88s 2ms/step - loss: 1.3299
Epoch 4/4
39641/39641 [==============================] - 88s 2ms/step - loss: 1.1100


In [20]:
truth=[np.argmax(value) for value in to_categorical(validation_set["Labels"])]
pred=[np.argmax(value) for value in model.predict(validation_set.drop("Labels",axis=1))]
result.append(accuracy(pred,truth))
print("\tVALIDATION ACCURACY : {}\n\n".format(accuracy(pred,truth)))

	VALIDATION ACCURACY : 0.057016663320618347




# ---------------------------------------------------------------------------------------------------------------

In [62]:
with open("word_embedder_20_new.pickle", "wb") as file:
    pickle.dump(word_embedder, file)
